In [ ]:
# Gerekli kütüphanelerin import edilmesi
import os, sys, glob, cv2, pdb, random
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import pylab as pl
from keras import backend as K
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras import *
from tensorflow.keras import models


In [ ]:
# Görüntülerin bulunduğu dizin ve sınıf etiketleri
imdir = "OvarianCancer/"
load_images_from_folder = ['Clear_Cell', 'Endometri', 'Mucinous', 'Non_Cancerous', 'Serous']

# Görüntü boyutu ve parlaklık artırma miktarı gibi önemli parametrelerin tanımlanması
f_size = (220, 220) # image dimension

# CLAHE parametrelerini tanımlayın
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# Görüntü veri setinin yüklenmesi ve hazırlanması
dt_set = [] 
imgs = []
labels = []
img_hsv = []
img_rgb = []
img_gray = []
images_=[]
img_ycbcr = []

# Görüntülerin yüklenmesi ve dönüştürülmesi
for i, load_img in enumerate(load_images_from_folder):
    path = os.path.join(imdir, load_img)
    class_number = i
    
    for j, image_name in enumerate(os.listdir(path)):
        image_path = os.path.join(path, image_name)
        dt_set.append((image_path, class_number))

In [ ]:
# Görüntülerin yüklenmesi ve dönüştürülmesi devam ediyor
for data in dt_set:
    image_path, load_img = data
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, f_size)
        
        img_ycbcr_temp = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        img_hsv_temp = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        img_gray_temp = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Görüntüler ve dönüşümleri listelere ekleniyor
        imgs.append(img)
        labels.append(load_img)
        img_hsv.append(img_hsv_temp)
        img_rgb.append(img)
        img_gray.append(img_gray_temp)
        img_ycbcr.append(img_ycbcr_temp)
        
# Dönüştürülmüş görüntüler listelerini NumPy dizilerine dönüştür
imgs = np.array(imgs, dtype=np.float32) / 255.0
labels = np.array(labels)
img_hsv = np.array(img_hsv)
img_rgb = np.array(img_rgb)
img_gray = np.array(img_gray)
img_ycbcr = np.array(img_ycbcr)

In [ ]:
preprocess_input=tf.keras.applications.vgg16.preprocess_input

In [ ]:
base_dir='OvarianCancer'
train_datagen=ImageDataGenerator(rescale=1./255, validation_split=0.3, preprocessing_function=preprocess_input)
test_datagen=ImageDataGenerator(rescale=1./255, validation_split=0.3, preprocessing_function=preprocess_input)

train_datagen=train_datagen.flow_from_directory(base_dir,target_size=(224,224),subset="training",batch_size=2,class_mode="sparse")
test_datagen=test_datagen.flow_from_directory(base_dir,target_size=(224,224),subset="validation",batch_size=2,class_mode="sparse")

In [ ]:
vgg_model=VGG16()

In [ ]:
vgg_model.summary()

In [ ]:
print(type(vgg_model))

In [ ]:
model=Sequential()
for layer in vgg_model.layers[0:-1] :
    model.add(layer)

In [ ]:
model.summary()


In [ ]:
for layer in model.layers:
    layer.trainable=False
model.summary()

In [ ]:
model.add(layers.Dense(5))
model.summary()

In [ ]:
optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer,loss=loss,metrics=["accuracy"])

In [ ]:
result=model.fit(train_datagen,epochs=35,verbose=1,validation_data=test_datagen)

In [ ]:
acc=result.history["accuracy"]
val_acc=result.history["val_accuracy"]
acc_loss=result.history["loss"]
val_loss=result.history["val_loss"]
epoch=range(1,len(acc)+1)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(epoch,acc,label=("egitimin basarisi"),color="b")
plt.plot(epoch,val_acc,label=("dogrulama basarisi"),color="r")
plt.title("egitim ve dogrulama basarisi")
plt.legend()

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(epoch,acc_loss,label=("egitimin basarisi"),color="b")
plt.plot(epoch,val_loss,label=("dogrulama basarisi"),color="r")
plt.title("egitim ve dogrulama kaybi")
plt.legend()

In [ ]:
# Adaptif eşikleme fonksiyonunun tanımlanması
def adaptive_thresholding(noisy_image):
    # Adaptif eşikleme işlemi
    _, adaptive_thresh = cv2.threshold(noisy_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    adaptive_thresh = cv2.adaptiveThreshold(noisy_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)
    
    # Adaptif eşikleme sonucunu döndür
    return adaptive_thresh

# Her bir görüntünün farklı renk uzaylarında ve işlenmiş hallerinin gösterilmesi
for i, data in enumerate(dt_set):
    image_path, load_img = data
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Her iterasyon için yeni bir figür oluşturulur
    plt.figure(figsize=(25, 10))
    
    # RGB, Grayscale ve YCbCr görsellerinin gösterimi
    plt.subplot(2, 5, 1)
    plt.imshow(img_rgb[i])
    plt.title(f"RGB / Boyut: {img_rgb[i].shape}")

    plt.subplot(2, 5, 2)
    plt.imshow(img_gray[i], cmap='gray')
    plt.title(f"Grayscale / Boyut: {img_gray[i].shape}")

    plt.subplot(2, 5, 3)
    plt.imshow(img_ycbcr[i], cmap='viridis')
    plt.title(f"YCbCr / Boyut: {img_ycbcr[i].shape}")
    
    # HSV ve eşitlenmiş HSV görsellerinin gösterimi
    plt.subplot(2, 5, 4)
    plt.imshow(img_hsv[i], cmap='plasma')
    plt.title(f"HSV / Boyut: {img_hsv[i].shape}")

    hsv_image = img_hsv[i].copy()
    h, s, v = cv2.split(hsv_image)
    equalized_v = cv2.equalizeHist(v)
    hsv_image = cv2.merge([h, s, equalized_v])
    plt.subplot(2, 5, 5)
    plt.imshow(cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB))
    plt.title(f"Equalized HSV / Boyut: {hsv_image.shape}")

    # Histogram eşitleme işlemi
    # Equalized V histogramı
    plt.subplot(2, 5, 6)
    plt.hist(equalized_v.flatten(), bins=50, color='r')
    plt.title('Equalized V Histogram')

    # BGR görüntüsü ve kenar tespiti (Canny) gösterimi
    plt.subplot(2, 5, 7)

    bgr_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)

    edges = cv2.Canny(gray_image, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_image = np.zeros_like(bgr_image)
    cv2.drawContours(contour_image, contours, -1, (255, 255, 255), thickness=cv2.FILLED)

    #renklendirme işlemi
    renkli_goruntu = cv2.applyColorMap(img_gray[i], cv2.COLORMAP_JET)
    
    # CLAHE işlemi
    clahe_output = clahe.apply(cv2.cvtColor(renkli_goruntu, cv2.COLOR_BGR2GRAY))
    
     # Gaussian gürültü ekleme işlemi
    mean = 0
    stddev = 25
    noise = np.zeros(clahe_output.shape, dtype=np.uint8)
    cv2.randn(noise, mean, stddev)
    noisy_image = cv2.add(clahe_output, noise)
    
    #adaptif eşikleme işlemi
    adaptive_thresh = adaptive_thresholding(noisy_image)

    # Görsellerin gösterimi
    plt.imshow(bgr_image)
    plt.title('BGR Image')

    plt.subplot(2, 5, 8)
    plt.imshow(edges, cmap='gray')
    plt.title('Edges')

    plt.subplot(2, 5, 9)
    plt.imshow(contour_image)
    plt.title('Contours')

    plt.subplot(2, 5, 10)
    plt.imshow(renkli_goruntu)
    plt.title('Renk Haritası Kullanarak Renklendirilmiş Grayscale Görüntü')

    plt.tight_layout()
    plt.show()

    # CLAHE ve Gaussian gürültü eklenmiş görüntülerin gösterimi
    plt.figure(figsize=(20, 10))
    
    plt.subplot(2, 4, 1)
    plt.imshow(clahe_output, cmap='plasma')
    plt.title('CLAHE Output')
    plt.axis('off')
    
    plt.subplot(2, 4, 2)
    plt.imshow(noisy_image, cmap='hot')
    plt.title('Noisy Image')
    plt.axis('off')

    plt.subplot(2, 4, 4)
    plt.imshow(adaptive_thresh, cmap='gray')
    plt.title('Adaptif Thresholding Sonucu')
    plt.show()
    
    plt.tight_layout()
    plt.show()
    print(f"Görüntü Adı: {dt_set[i][0]}, Boyutu: {img_rgb[i].shape}")